In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer as si
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np
import pandas as pd

In [32]:
def read_data(file_path):
    data = pd.read_csv(file_path, header=None)
    x = pd.DataFrame(data)
    x = x.replace('?',np.nan)
    model = si(missing_values=np.nan ,strategy='most_frequent')
    x = model.fit_transform(x).astype(np.int)
    return x

In [38]:
train = read_data('train.csv')
test = read_data('test.csv')

model = SelectKBest(chi2, k=6)
model.fit(train[:,:-1], train[:,-1])

train_x = model.transform(train[:,:-1])
train_y = train[:,-1]

In [41]:
x1 = train_x[:-1000]
y1 = train_y[:-1000]
x2 = train_x[-1000:]
y2 = train_y[-1000:]

In [42]:
knn = KNeighborsClassifier(n_neighbors=32)
knn.fit(x1,y1)
knn.score(x2,y2)

0.32000000000000001

In [50]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, include_bias = False)
poly.fit(x1)
X_poly1 = poly.transform(x1)
X_poly2 = poly.transform(x2)

In [52]:
X_poly2.shape

(1000, 83)

In [56]:
import torch
import torch.nn as nn

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.line = nn.Sequential(
            nn.Linear(83,1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024,512),            
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,9)
        )
    
    def forward(self, x):
        out = self.line(x)
        return out

In [73]:
train_x = torch.from_numpy(X_poly1).float()
train_y = torch.from_numpy(y1-1)

net = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())
out = net(train_x)
for i in range(1000):
    out = net(train_x)
    loss = loss_fn(out, train_y.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('%d/100 Loss: %.4f' % (i/100,loss.item()))

0/100 Loss: 11.0513
1/100 Loss: 1.8210
2/100 Loss: 1.7531
3/100 Loss: 1.7178
4/100 Loss: 1.6811
5/100 Loss: 1.6463
6/100 Loss: 1.6155
7/100 Loss: 1.5817
8/100 Loss: 1.5445
9/100 Loss: 1.5270


In [74]:
test_x = torch.from_numpy(X_poly2).float()
test_y = torch.from_numpy(y2-1)
pred = net(test_x)
_,pred = torch.max(pred,1)

In [75]:
q = (pred-test_y)

In [76]:
sum(q==0)

tensor(331)